# DSL Winter Project - RandomForestRegressor

The following file, unlike the other "ExtraTreesSolution" provided, contains also the codes used to generate all the graphs presented in the report.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

In [ ]:
# setting the random state
rs = 42

In [ ]:
df_full = pd.read_csv("development.csv")
df_full.shape

## Preprocessing

#### Domain constraints

In [ ]:
for i in range(18):
    mask = df_full[f'negpmax[{i}]']>0
    df_full = df_full.loc[~mask]

df_full.shape

### Detecting noise features

In [ ]:
groups = []
for i in range(18):
    groups.append([f'pmax[{i}]', f'negpmax[{i}]', f'area[{i}]', f'tmax[{i}]', f'rms[{i}]'])

In [ ]:
df_pca = df_full.copy()

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

for i, g in enumerate(groups):
    group_features = df_pca[g]
    
    # standardize the features within the group
    scaler = StandardScaler()
    st_group_features = scaler.fit_transform(group_features)
    
    # apply PCA
    pca = PCA(n_components=1)
    pca_result = pca.fit_transform(st_group_features)
    
    # insert pca result into the copy of the df
    df_pca[f"pad_{i}"] = pca_result

In [ ]:
pads = [ f'pad_{i}' for i in range(18)]
sel = ['x', 'y']+pads

df_pca = df_pca.loc[:,sel]

In [ ]:
custom_cmap = sns.diverging_palette(145, 300, s=60, as_cmap=True)
sns.heatmap(df_pca.corr(), cmap=custom_cmap)
plt.savefig('pads_correlation.pdf', bbox_inches='tight')

In [ ]:
noise_columns = [f"pmax[{i}]" for i in [0, 7, 12, 15, 16, 17]] + \
                [f"negpmax[{i}]" for i in [0, 7, 12, 15, 16, 17]] + \
                [f"area[{i}]" for i in [0, 7, 12, 15, 16, 17]] + \
                [f"tmax[{i}]" for i in [0, 7, 12, 15, 16, 17]] + \
                [f"rms[{i}]" for i in [0, 7, 12, 15, 16, 17]]
noise_columns
df_nonoise = df_full.drop(columns=noise_columns)

In [ ]:
df_nonoise['x'].corr(df_nonoise['y'])

### Investigating the features' importance

In [ ]:
fig, axes = plt.subplots(3,4, figsize=(8,8))
axes = axes.flatten()

for i, ax in zip([1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14], axes):
    sc = ax.scatter(df_nonoise['x'], df_nonoise['y'], c=df_nonoise[f'pmax[{i}]'].values, alpha=0.2)
    ax.set_title(f'pmax[{i}]',fontsize=10)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.tight_layout()
plt.savefig('pmax_scatterbigger.png', bbox_inches='tight')
plt.show()

In [ ]:
corr_mat = df_nonoise.iloc[:,2:].corr()

# correlation threshold
corr_th = 0.9

corr_pairs = []

for i in range(len(corr_mat.columns)):
    for j in range(i+1, len(corr_mat.columns)):
        if abs(corr_mat.iloc[i, j]) > corr_th:
            corr_pairs.append((corr_mat.columns[i], corr_mat.columns[j]))

# Print the correlated variable pairs
for pair in corr_pairs:
    print(f"Correlation between {pair[0]} and {pair[1]}: {corr_mat.loc[pair[0], pair[1]]}")

In [ ]:
df = df_nonoise
df.shape

In [ ]:
# dividing df in X (inputs) and y (target variables)
y = df.loc[:,["x", "y"]]
X = df.iloc[:,2:]

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, stratify=y, random_state=rs)

In [ ]:
# definition of the evaluation metric
def euclidean_metric(y_true, y_pred):
    return np.mean(np.sqrt(np.sum((y_true-y_pred)**2, axis=1)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf0 = RandomForestRegressor(n_estimators=100, random_state=rs, n_jobs=-1, verbose=5)
rf0.fit(X_train, y_train)
print(f"r2: {r2_score(y_test, rf0.predict(X_test))} avg_euclidean:{euclidean_metric(y_test, rf0.predict(X_test))}")

In [ ]:
feature_names = X.columns
sorted(zip(feature_names, rf0.feature_importances_), key=lambda x: x[1], reverse=True)
# It seems that tmax and rms are not important features

In [ ]:
plt.figure(figsize=(8, 13))
feature_importance = pd.Series(rf0.feature_importances_, index = feature_names)
feature_importance_sorted = feature_importance.sort_values(ascending=True)
feature_importance_sorted.plot(kind='barh', logx=True)
plt.yticks(fontsize=10)
plt.xlabel('Importance (log scale)') # --> in order to make results more readable on the graph
plt.tight_layout()
plt.savefig('feature_importancelog.pdf', bbox_inches='tight')

In [ ]:
r = [f"rms[{i}]" for i in range(18) if i not in [0, 7, 12, 15, 16, 17]]
t = [f"tmax[{i}]" for i in range(18) if i not in [0, 7, 12, 15, 16, 17]]
# Remove rms and tmax
c = r + t
X_train, X_test, y_train, y_test = train_test_split(X.drop(columns=c), y, test_size=0.20, stratify=y, shuffle=True, random_state=rs)
X_train

## Validation

##### Trend of the performance in relation to n_estimators

In [ ]:
# We took inspiration from the code provided on the website " https://www.kaggle.com/code/ahmedabdulhamid/best-n-estimators-for-randomforest " to plot the following graph.

predictions = []
for tree in rf1.estimators_:
    predictions.append(tree.predict(X_test.values)[None, :])

predictions = np.vstack(predictions)
cum_mean = np.cumsum(predictions, axis=0)/np.arange(1, predictions.shape[0] + 1)[:, None, None]
scores = []
for pred in cum_mean:
    scores.append(euclidean_metric(y_test, pred))

plt.figure(figsize=(10, 6))
plt.plot(scores, linewidth=3)
plt.xlabel('Number of trees')
plt.grid()
plt.ylabel('Average euclidean distance')
plt.savefig('num_of_trees.pdf', bbox_inches='tight')

### RF - GridSearch

Although we had already defined a validation test, we will now conduct a cross-validation, in order to be sure that we do not overfit a single subset of the _development_ set.

In [ ]:
import time
from datetime import datetime
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

params_rf = {'n_estimators': [100, 200, 300],
                  'max_features': ["sqrt", 0.33, 0.5],
                  'criterion': ["squared_error", "poisson"],
                  'n_jobs': [-1],
                  'max_depth': [None, 10, 30, 50],
                  'random_state': [rs]
            }


print(f"start: {datetime.now()}")
gs_rf = GridSearchCV(RandomForestRegressor(), params_rf, scoring=make_scorer(euclidean_metric, greater_is_better=False),
                          n_jobs=-1, cv=3, verbose=5)
gs_rf.fit(X_train, y_train)
print(f"end: {datetime.now()}")

In [ ]:
gs_rf.best_params_

{'criterion': 'squared_error', 'max_depth': None, 'max_features': 0.33, 'n_estimators': 300, 'n_jobs': -1, 'random_state': 42}

In [ ]:
gs_rf.best_score_
# -4.079161974683277

In [ ]:
gs_rf.cv_results_

## Results

### Result on test set

In [ ]:
print(f"r2: {r2_score(y_test, gs_rf.predict(X_test))} avg_euclidean:{euclidean_metric(y_test, gs_rf.predict(X_test))}") #---> optimal configuration

### Conclusions 

**Final RandomForestRegressor**

In [ ]:
df_eval = pd.read_csv("evaluation.csv")

In [ ]:
df_eval.drop(columns=noise_columns+c+['Id'], inplace=True) # drop noise, tmax, rms

In [ ]:
X_full = df_full.iloc[:,2:]
y_full = df_full.loc[:,['x','y']]

In [ ]:
X_full = X_full.drop(columns=noise_columns+c) # drop noise, tmax, rms

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

final_rf = RandomForestRegressor(n_estimators=300, max_features=0.33, criterion = 'squared_error', max_depth = None, random_state=rs, n_jobs=-1) #---> optimal configuration

print(f"start fitting: {datetime.now()}")
final_rf.fit(X_full, y_full)
print(f"end fitting: {datetime.now()}")

print(f"start predicting: {datetime.now()}")
predictions = final_rf.predict(df_eval)
print(f"end predicting: {datetime.now()}")

data = {'Id': np.arange(0, predictions.shape[0]), 'Predicted': [f"{val[0]}|{val[1]}" for val in predictions]}
submission = pd.DataFrame(data)
submission.to_csv("output_rfFINAL.csv", index=False)